# AI Tarot Music Project

In [1]:
# Import necessary libraries
import os
import requests
from dotenv import load_dotenv
print ("All imports successful")

All imports successful


In [2]:
# Load environment variables from .env file
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if GROQ_API_KEY is None:
    raise ValueError("GROQ API key not found.")


In [3]:
# Define the API endpoint and headers
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

In [4]:
# Function to build the prompt for the API call
def build_prompt(cards, context):
    return f"""\n
You are a reflective assistant combining tarot symbolism and music psychology.\n
You do not predict the future. You provide grounded, introspective interpretations.\n\n
Tarot cards drawn:\n
{', '.join(cards)}\n\n
Context:\n{context}\n\n
Tasks:\n
1. Interpret the tarot reading\n
2. Identify emotional themes\n
3. Recommend 3 music genres or styles\n
4. Explain why each recommendation fits\n\n
Respond clearly and thoughtfully.\n
"""

In [5]:
# Example usage
tarot_cards = ["The Hermit", "The Tower"]
user_context = "Career"

prompt = build_prompt(tarot_cards, user_context)

In [6]:
# Prepare the payload for the API request
payload = {
    "model": "llama-3.1-8b-instant",  # or another Llama 3 variant
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "max_tokens": 512,
    "temperature": 0.7
}

In [7]:
# Send the request to Groq API and print the result
response = requests.post(GROQ_API_URL, headers=headers, json=payload)
if response.status_code == 200:
    result = response.json()
    # The response format may vary; adjust as needed
    print(result.get('choices', [{}])[0].get('message', {}).get('content', 'No content found.'))
else:
    print(f"Error: {response.status_code} - {response.text}")

Reflecting on the tarot reading, I see a dynamic interplay between The Hermit and The Tower. Here's my interpretation:

**The Hermit:** The Hermit suggests a need for introspection, self-reflection, and withdrawal from external stimuli. In the context of career, this card may indicate that you're feeling the need to retreat, recharge, and focus on your inner guidance. This might be a call to slow down, reassess your priorities, and listen to your inner voice. The Hermit can also represent a time of solitude, where you're being asked to explore your own thoughts, values, and goals.

**The Tower:** The Tower, on the other hand, represents sudden change, upheaval, and transformation. In the context of career, this card may indicate a need to break free from restrictive patterns, shake off complacency, or confront unresolved issues that have been holding you back. The Tower can also symbolize a sudden revelation, a wake-up call, or a major shift in perspective that forces you to re-evaluat

In [9]:
import json
from IPython.display import display, HTML

# --- Step 1: Extract genres from the LLM response using Groq ---
extract_payload = {
    "model": "llama-3.1-8b-instant",
    "messages": [
        {
            "role": "user",
            "content": (
                "From the following text, extract ONLY the music genre/style names as a JSON list of strings. "
                "Return ONLY the JSON array, nothing else.\n\n"
                + result['choices'][0]['message']['content']
            )
        }
    ],
    "max_tokens": 100,
    "temperature": 0.0
}

genre_response = requests.post(GROQ_API_URL, headers=headers, json=extract_payload)
raw = genre_response.json()['choices'][0]['message']['content'].strip()
genres = json.loads(raw)
print("Extracted genres:", genres)

# --- Step 2: Get Spotify Access Token (Client Credentials Flow) ---
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

token_url = "https://accounts.spotify.com/api/token"
token_response = requests.post(token_url, data={
    "grant_type": "client_credentials",
    "client_id": SPOTIFY_CLIENT_ID,
    "client_secret": SPOTIFY_CLIENT_SECRET,
})
access_token = token_response.json()["access_token"]
spotify_headers = {"Authorization": f"Bearer {access_token}"}
print("Spotify authentication successful!")

# --- Step 3: Search for tracks per genre ---
SPOTIFY_SEARCH_URL = "https://api.spotify.com/v1/search"
all_tracks = []

for genre in genres:
    params = {
        "q": f"genre:{genre}",
        "type": "track",
        "limit": 3
    }
    search_resp = requests.get(SPOTIFY_SEARCH_URL, headers=spotify_headers, params=params)
    tracks = search_resp.json().get("tracks", {}).get("items", [])

    if not tracks:
        params["q"] = genre
        search_resp = requests.get(SPOTIFY_SEARCH_URL, headers=spotify_headers, params=params)
        tracks = search_resp.json().get("tracks", {}).get("items", [])

    for t in tracks:
        all_tracks.append(t)
        print(f"[{genre}] {t['name']} — {t['artists'][0]['name']}")

# --- Step 4: Display embedded Spotify players in the notebook ---
if all_tracks:
    html_parts = []
    for t in all_tracks:
        track_id = t["id"]
        name = t["name"]
        artist = t["artists"][0]["name"]
        embed_url = f"https://open.spotify.com/embed/track/{track_id}?utm_source=generator&theme=0"
        html_parts.append(
            f'<p><strong>{name}</strong> — {artist}</p>'
            f'<iframe src="{embed_url}" width="100%" height="80" '
            f'frameBorder="0" allow="autoplay; clipboard-write; encrypted-media; fullscreen; picture-in-picture" '
            f'loading="lazy"></iframe>'
        )
    display(HTML("\n".join(html_parts)))
else:
    print("No tracks found.")

Extracted genres: ['Electronic/Synthwave', 'Ambient/Instrumental', 'Hard Rock/Progressive']
Spotify authentication successful!
[Electronic/Synthwave] Heaven Falls — Synthwave Siren
[Electronic/Synthwave] Nightcall — Kavinsky
[Electronic/Synthwave] Kids — Kyle Dixon & Michael Stein
[Ambient/Instrumental] Catchacoma Lake — Little Symphony
[Ambient/Instrumental] I've Learned to Lean — Holy Communion Instrumental Duo
[Ambient/Instrumental] Yellowstone — Little Symphony
[Hard Rock/Progressive] Big Mistake — Anthony Green
[Hard Rock/Progressive] Leviathan — Yngwie Malmsteen
[Hard Rock/Progressive] Neon Knights — Queensrÿche
